<h2>Final Exam</h2>
You are a stock analyst working in the research department of a small<br>
hedge fund.  Your job is to analyze the returns of a pool of securities<br>
and back test the quants strategy of buys and sells.  You need<br>
to load the 4 securities and test their strategy.<br>
files:<br>
INTC - Intel Corporation<br>
AMD - Advanced Micro Devices<br>
AAPL - Apple<br>
FB - Meta Platforms, Inc.<br>
corporations = {'INTC': 'Intel Corporation',<br>
'AMD':'Advanced Micro Devices',<br>
'AAPL':'Apple',<br>
'FB':'Meta Platforms, Inc.'}<br>
<br>
NOTE: All trades use the closing price!!!!

<h2>Step 1</h2>
<h4>5 points</h4>
Create a directory and place all of the csv files into it<br>
Create the variable proj_dir = '  <your directory> '<br>
db_name = 'stock_analysis'

In [674]:
proj_dir = '/Users/DevOps/Documents/code-immersives/python/term-1/SQL_DB/Final_Files/'
db_name = 'Stock_Analysis'

<h2>Step 2</h2>
<h4>5 points</h4>
Create a database called 'Stock_Analysis'

In [675]:
import os
import sqlite3

os.chdir('/Users/DevOps/Documents/code-immersives/python/term-1/SQL_DB/')
conn = sqlite3.connect('Stock_Analysis.db')

<h2>Step 3</h2>
<h4>15 points</h4>
Create the following tables:<br>
1 - security_master<br>
symbol<br>
trade_date<br>
open_price<br>
high_price<br>
low_price<br>
adj_close<br>
close_price<br>
volume<br>
<br>
2 - security_info<br>
symbol<br>
name  Max of 25 characters<br>
research_notes Max of 255 characters<br>
<br>
3 - security_log<br>
transaction_id  Character max 20 alpha numeric<br>
trade_date<br>
symbol<br>
trans_type   - (will contain only 'BUY' or 'SELL')<br>
shares   (Will only contain whole numbers)<br>
cost - Will hold dollar amounts<br>
balance - Will hold dollar amounts<br>

In [676]:
c = conn.cursor()

c.executescript('''CREATE TABLE IF NOT EXISTS security_master(
symbol TEXT,
trade_date TEXT,
open_price REAL,
high_price REAL,
low_price REAL,
adj_close REAL,
close_price REAL,
volume INTEGER
);
DELETE FROM security_master;
CREATE TABLE IF NOT EXISTS security_info(
symbol TEXT,
name TEXT,
research_notes TEXT
);
DELETE FROM security_info;
CREATE TABLE IF NOT EXISTS security_log(
transaction_id TEXT,
trade_date TEXT,
symbol TEXT,
trans_type TEXT,
shares INTEGER,
cost REAL,
balance REAL
);
DELETE FROM security_log;
''')
conn.commit()

<h2>Step 4</h2>
<h4>15 points</h4>
Load the security master table<br>
Your code should read whatever files that are csv files in the directory<br>
Use the dictionary below to populate the security_info table:<br>
corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

In [677]:
import glob

proj_dir = '/Users/DevOps/Documents/code-immersives/python/term-1/SQL_DB/Final_Files/'

for file in glob.glob(proj_dir+ "*.csv"):
	fileParts = file.split('/')
	with open(proj_dir+fileParts[-1], 'r') as fh:
		line = fh.readline()
		counter = 0
		while line:
			if counter > 0:
				date, opening, high, low, close, adjClose, volume = line.replace('\n','').split(',')
				c.execute('INSERT INTO security_master VALUES (?,?,?,?,?,?,?,?)', (fileParts[-1][:-4], date, opening, high, low, adjClose, close, volume))
				conn.commit()
			counter += 1
			line = fh.readline()

corporations = {'INTC': 'Intel Corporation', 'AMD': 'Advanced Micro Devices',
                'AAPL': 'Apple', 'FB': 'Meta Platforms, Inc.'}
corpData = []

for i,j in corporations.items():
	corpData.append((i,j,''))

c.executemany('INSERT INTO security_info VALUES (?,?,?)', corpData)
conn.commit()


<h2>Step 5</h2>
<h4>10 points</h4>
Print the symbol,name, and number of records of every security using 1 SQL statement<br>

In [678]:
c.execute('''SELECT B.name, A.symbol, count(*) FROM security_master AS A
	LEFT JOIN security_info B ON A.symbol = B.symbol
	GROUP BY A.symbol
''')
print(c.fetchall())

[('Apple', 'AAPL', 754), ('Advanced Micro Devices', 'AMD', 754), ('Meta Platforms, Inc.', 'FB', 754), ('Intel Corporation', 'INTC', 754)]


<h2>Step 6</h2>
<h4>10 points</h4>
Print the average closing share price for each security in the year 2019<br>

In [679]:
c.execute('''SELECT symbol, avg(close_price) FROM security_master WHERE SUBSTR(trade_date,1,4) = '2019'
	GROUP BY symbol
''')
print(c.fetchall())

[('AAPL', 52.06398806349207), ('AMD', 29.94107146031748), ('FB', 181.63749959523804), ('INTC', 51.37519813492058)]


<h2>Step 7</h2>
<h4>5 points</h4>
Print the max and min closing share price for each security in the year 2020<br>

In [680]:
c.execute('''SELECT symbol, min(close_price), max(close_price) FROM security_master WHERE SUBSTR(trade_date,1,4) = '2020'
	GROUP BY symbol
''')
print(c.fetchall())

[('AAPL', 56.092499, 136.690002), ('AMD', 38.709999, 97.120003), ('FB', 146.009995, 303.910004), ('INTC', 44.110001, 68.470001)]


<h2>Step 8</h2>
<h4>15 points</h4>
Create a trading strategy to update the security_log table<br>
if closed "up" for 5 consecutive days SELL 10000 shares, sell if "down" for 5 consecutive days <br>
transaction_id = YYYYMMDD_symbol_random number, 100 up to 999<br>
trade_date <br>
symbol <br>
trans_type BUY OR SELL<br>
shares number of shares<br>
cost =  -1 * close_price * shares if a BUY, close_price * shares if a SELL<br>
<br>

In [681]:
import random
import datetime

c.execute(''' SELECT symbol, trade_date, close_price - open_price as diff, close_price FROM security_master WHERE symbol = 'AMD';''')
rows = c.fetchall() 

up_days = 0; down_days = 0; tot_val = 0; tot_shares = 0; share_size = 10000

for row in rows:
	symbol = row[0]
	myDate = datetime.datetime((int(row[1][:4])), int(row[1][5:7]), int(row[1][8:]))
	y, m, d = str(myDate.year), ('00'+str(myDate.month))[-2:], ('00'+str(myDate.day))[-2:]
	transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}' 
	trade_date = row[1]
	''' if closed up for 5 consecutive days SELL 10000 shares sell if down for 5 consecute days ''' 
	if row[2] > 0: 
		up_days += 1 
		down_days = 0 
	else: 
		up_days = 0 
		down_days += 1 
	if up_days >= 5: 
		down_days = 0 
		'''SELL signal''' 
	if tot_shares > 0: 
		factor = 2 if tot_shares - (share_size*2) > 0 else 1 
		tot_val += ((share_size*factor) * row[3]) 
		tot_shares -= (share_size*factor)  
		c.execute('''INSERT INTO security_log VALUES(?,?,?,?,?,?,?)''', (transaction_id, trade_date, symbol, "SELL", (share_size*factor), ((share_size*factor) * row[3]), tot_val))
		conn.commit()
		print(f'****** SELL HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')
	if down_days >= 5:
			up_days = 0
			down_days = 0
			'''buy signal'''
			tot_val += (-1 * share_size * row[3])
			tot_shares += share_size
			c.execute('''INSERT INTO security_log VALUES(?,?,?,?,?,?,?)''', (transaction_id, trade_date,
			          symbol, "BUY", (share_size*factor), ((share_size*factor) * row[3]), tot_val))
			conn.commit()
			print(
					f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')


****** BUY HERE AMD - 2019-02-07 - 22.67 - 10000 - -226700.00000000003
****** SELL HERE AMD 2019-02-08 23.049999 - 0 3799.9899999999616
230499.99
****** BUY HERE AMD - 2019-03-28 - 25.059999 - 10000 - -246800.00000000006
****** SELL HERE AMD 2019-03-29 25.52 - 0 8399.999999999942
255200.0
****** BUY HERE AMD - 2019-06-25 - 28.860001 - 10000 - -280200.01000000007
****** SELL HERE AMD 2019-06-26 29.92 - 0 18999.989999999932
299200.0
****** BUY HERE AMD - 2019-08-06 - 28.860001 - 10000 - -269600.0200000001
****** SELL HERE AMD 2019-08-07 29.190001 - 0 22299.989999999932
291900.01
****** BUY HERE AMD - 2020-05-01 - 49.880001 - 10000 - -476500.0200000001
****** SELL HERE AMD 2020-05-04 52.560001 - 0 49099.98999999993
525600.01
****** BUY HERE AMD - 2020-10-20 - 81.559998 - 10000 - -766499.99
****** SELL HERE AMD 2020-10-21 79.199997 - 0 25499.97999999998
791999.97
****** BUY HERE AMD - 2020-12-22 - 93.160004 - 10000 - -906100.06
****** SELL HERE AMD 2020-12-23 91.550003 - 0 9399.96999999997

<h2>Step 9</h2>
<h4>10 points</h4>
Return the symbol,stock price,trans_type For all 2020 trades<br>

In [682]:
c.execute('''SELECT symbol, cost, trans_type FROM security_log WHERE SUBSTR(trade_date,1,4) = '2020' ''')
print(c.fetchall())


[('AMD', 498800.01, 'BUY'), ('AMD', 525600.01, 'SELL'), ('AMD', 815599.98, 'BUY'), ('AMD', 791999.97, 'SELL'), ('AMD', 931600.04, 'BUY'), ('AMD', 915500.03, 'SELL')]


<h2>Step 10</h2>
<h4>10 points</h4>
If you bought 100,000 shares of FB the first trading day of 2020 and sold<br>
100,000 shares on the last day of 2020. How much would you have gained or lost?

In [683]:
c.execute('''SELECT max(trade_date), 100000*close_price FROM security_master WHERE SUBSTR(trade_date,1,4) = '2020' AND symbol = 'FB'
	UNION
	SELECT min(trade_date), 100000*close_price FROM security_master WHERE SUBSTR(trade_date,1,4) = '2020' AND symbol = 'FB'
''')
row = c.fetchall()
print(row)
print(f'${round(row[1][1] - row[0][1],2)} gained if number is positive.')


[('2020-01-02', 20977999.9), ('2020-12-31', 27316000.400000002)]
$6338000.5 gained if number is positive.
